## Importing

In [ ]:
!pip install transformers -q
!pip install texthero -q
!pip install tensorflow_text -q

     |████████████████████████████████| 241 kB 7.6 MB/s 
     |████████████████████████████████| 1.5 MB 10.5 MB/s 


In [ ]:
import os
import csv
import time
import warnings

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import transformers
import tensorflow_hub as hub
import tensorflow_text as text

from texthero import clean
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

## Config

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.122.58.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.122.58.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
BATCH_SIZE = 256
MAX_LEN = 250

## Loading data

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/AmazonML/input/new_train.csv', 
                    usecols=['TITLE', 'BULLET_POINTS', 'label3', 'split3'])
train.fillna('nan', inplace=True)

num_classes = train['label3'].nunique()

## Prepraing dataset

In [ ]:
le = LabelEncoder()
train['label3'] = le.fit_transform(train['label3'])

train_df, valid_df = train[train['split3'] == 'train'], train[train['split3'] == 'valid']
train_df['text'] = train_df['TITLE'] + " " + train_df['BULLET_POINTS']
valid_df['text'] = valid_df['TITLE'] + " " + valid_df['BULLET_POINTS']

train_text = train_df['text'].values
valid_text = valid_df['text'].values

train_label = train_df['label3'].values
valid_label = valid_df['label3'].values

In [ ]:
tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-uncased')

## Creating model

In [ ]:
def create_model():
    bert = transformers.TFBertModel.from_pretrained('bert-base-uncased', 
                                                    output_attentions=False, 
                                                    output_hidden_states=False)
    inputs = tf.keras.layers.Input((None,), dtype=tf.int32)
    mask = tf.keras.layers.Input((None,), dtype=tf.int32)
    bert_outputs = bert(inputs, attention_mask=mask)[1]
    preds = tf.keras.layers.Dense(num_classes, activation='softmax')(bert_outputs)
    return tf.keras.Model([inputs, mask], preds)

## Training Function

In [ ]:
print('creating model')
with strategy.scope():
  model = create_model()
  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
  loss_fun = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss_fun, metrics=['acc'])

creating model


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [ ]:
def train_chunk(start, end):
  
  valid_idx = np.random.randint(0, len(valid_text)-1, size=25000)
  
  valid_text_chunk = list(valid_text[valid_idx])
  train_text_chunk = list(train_text[start:end])

  train_label_chunk = train_label[start:end]
  valid_label_chunk = valid_label[valid_idx]

  start = time.time()
  print('Tokenizing data')
  train_data = tokenizer(train_text_chunk, padding=True, truncation=True, max_length=MAX_LEN)
  valid_data = tokenizer(valid_text_chunk, padding=True, truncation=True, max_length=MAX_LEN)
  print('Tokenization complete, time taken = ', time.time() - start)

  start = time.time()
  print('creating dataset')
  train_dataset = tf.data.Dataset.from_tensor_slices(((train_data['input_ids'], 
                                                       train_data['attention_mask']), 
                                                       train_label_chunk)).batch(BATCH_SIZE)

  valid_dataset = tf.data.Dataset.from_tensor_slices(((valid_data['input_ids'], 
                                                       valid_data['attention_mask']), 
                                                       valid_label_chunk)).batch(BATCH_SIZE)
  print('dataset created, time taken = ', time.time() - start)
  
  print('TRAINING')
  model.fit(train_dataset,
            epochs=2,
            validation_data=valid_dataset)

  print('Saving weights')
  model.save_weights('/content/drive/MyDrive/AmazonML/tpu_model.h5')                                            

## Training the model

In [ ]:
len(train_df)

2123582

In [ ]:
start = 0
for i in range(100000, 2000000, 100000):
  train_chunk(start, i)
  start = i

Tokenizing data
Tokenization complete, time taken =  15.30895471572876
creating dataset
dataset created, time taken =  163.3544795513153
TRAINING
Epoch 1/2


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 250) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 250) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None,) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 250) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 250) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None,) dtype=int64>]


391/391 [==============================] - ETA: 0s - loss: 5.2346 - acc: 0.2836

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 250) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 250) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None,) dtype=int64>]


391/391 [==============================] - 327s 624ms/step - loss: 5.2346 - acc: 0.2836 - val_loss: 3.9856 - val_acc: 0.4173
Epoch 2/2
391/391 [==============================] - 175s 447ms/step - loss: 3.4518 - acc: 0.4846 - val_loss: 3.0809 - val_acc: 0.5250
Saving weights
Tokenizing data
Tokenization complete, time taken =  18.140333890914917
creating dataset
dataset created, time taken =  163.2547299861908
TRAINING
Epoch 1/2
391/391 [==============================] - 175s 447ms/step - loss: 2.8414 - acc: 0.5524 - val_loss: 2.5296 - val_acc: 0.5831
Epoch 2/2
391/391 [==============================] - 175s 447ms/step - loss: 2.3434 - acc: 0.6116 - val_loss: 2.2239 - val_acc: 0.6237
Saving weights
Tokenizing data
Tokenization complete, time taken =  17.59330940246582
creating dataset
dataset created, time taken =  162.4943528175354
TRAINING
Epoch 1/2
391/391 [==============================] - 175s 447ms/step - loss: 2.1752 - acc: 0.6264 - val_loss: 1.9986 - val_acc: 0.6463
Epoch 2/2
39

## Predicting on test set

In [ ]:
model.load_weights('/content/drive/MyDrive/AmazonML/tpu_model.h5')
test = pd.read_csv('/content/drive/MyDrive/AmazonML/input/dataset/test.csv', escapechar='\\', quoting=csv.QUOTE_NONE)
test.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [ ]:
test.fillna('nan', inplace=True)
len(test)

110775

In [ ]:
test_text = test['TITLE'] + " " + test['BULLET_POINTS']

In [ ]:
test_data = tokenizer(list(test_text), padding=True, truncation=True, max_length=MAX_LEN)

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(((test_data['input_ids'], 
                                                    test_data['attention_mask']), test['PRODUCT_ID'].values)).batch(BATCH_SIZE)

In [ ]:
product_ids = []
sample_preds = []
for i in tqdm(test_dataset):
  product_ids += list(i[1].numpy())
  sample_preds += list(np.argmax(model.predict(i[0]), axis=1))

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/AmazonML/input/new_train.csv', 
                    usecols=['label3'])
train.fillna('nan', inplace=True)

In [ ]:
le = LabelEncoder()
transformed = le.fit_transform(train['label3'])

In [ ]:
inverse = {}

for i, j in zip(transformed, train['label3']):
  inverse[i] = j

In [ ]:
# Temp sub
sub = pd.DataFrame()
sub['PRODUCT_ID'] = product_ids
sub['BROWSE_NODE_ID'] = sample_preds
sub['BROWSE_NODE_ID'] = sub['BROWSE_NODE_ID'].map(inverse)

In [ ]:
sub.to_csv('/content/drive/MyDrive/AmazonML/BertBaselineSubmission.csv', index=False)

In [ ]:
sub.head()

,PRODUCT_ID,BROWSE_NODE_ID
0,1,1140
1,2,15772
2,3,4821
3,4,125
4,5,8915
